In [3]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

my_url = 'https://www.projekt-gutenberg.org/fallada/hoppelpo/chap001.html'

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'html.parser')

In [11]:
#containers = page_soup.find_all('h3', {'class': 'qMZa55 SQGpu8 iOzucJ JT3_zV DvypSJ'})
container = page_soup.find_all('body')

In [17]:
container = container[0]

In [18]:
title = container.find_all('h3')

In [19]:
title

[<h3>Lieber Hoppelpoppel – wo bist du?</h3>]